In [24]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [25]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
         def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy')>DESIRED_ACCURACY:
                self.model.stop_training=True

callbacks = myCallback()

--2020-11-29 18:28:30--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-11-29 18:28:30 (132 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [26]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150, 150),
        batch_size=10,
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [28]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit_generator(
          train_generator, epochs=20, steps_per_epoch=4, verbose=1, callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
4/4 [==============================] - 1s 343ms/step - loss: 10.1558 - accuracy: 0.4750
Epoch 2/20
4/4 [==============================] - 1s 350ms/step - loss: 0.7499 - accuracy: 0.5750
Epoch 3/20
4/4 [==============================] - 1s 347ms/step - loss: 0.6595 - accuracy: 0.8250
Epoch 4/20
4/4 [==============================] - 1s 351ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 5/20
4/4 [==============================] - 1s 350ms/step - loss: 0.4393 - accuracy: 0.8000
Epoch 6/20
4/4 [==============================] - 1s 353ms/step - loss: 0.3336 - accuracy: 0.8000
Epoch 7/20
4/4 [==============================] - 1s 351ms/step - loss: 0.2099 - accuracy: 0.9000
Epoch 8/20
4/4 [==============================] - 1s 347ms/step - loss: 0.1902 - accuracy: 0.9250
Epoch 9/20
4/4 [==============================] - 1s 354ms/step - loss: 0.0926 - accuracy: 0.9750
Epoch 10/20
4/4 [==============================] - 1s 348ms/step - loss: 0.4110 - accuracy: 0.7500
Epoch 11/20
4/4 [=